---

📌 **This notebook has been updated [here](https://github.com/jhj0517/Whisper-WebUI.git)!**

🖋 **Author**: [jhj0517](https://github.com/jhj0517/Whisper-WebUI/blob/master/notebook/whisper-webui.ipynb)

😎 **Support the Project**:

If you find this project useful, please consider supporting it:

<a href="https://ko-fi.com/jhj0517" target="_blank">
    <img src="https://storage.ko-fi.com/cdn/kofi2.png?v=3" alt="Buy Me a Coffee at ko-fi.com" height="36">
</a>

---

In [1]:
#@title #(Optional) Check GPU
#@markdown Some models may not function correctly on a CPU runtime.

#@markdown so you should check your GPU setup before run.
!nvidia-smi

Sun Apr 13 10:10:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title #Installation
#@markdown This cell will install dependencies for Whisper-WebUI!
!git clone https://github.com/jhj0517/Whisper-WebUI.git
%cd Whisper-WebUI
!pip install git+https://github.com/jhj0517/jhj0517-whisper.git
!pip install faster-whisper==1.1.1
!pip install gradio
!pip install gradio-i18n
# Temporal bug fix from https://github.com/jhj0517/Whisper-WebUI/issues/256
!pip install git+https://github.com/JuanBindez/pytubefix.git
!pip install pyannote.audio==3.3.1
!pip install git+https://github.com/jhj0517/ultimatevocalremover_api.git

Cloning into 'Whisper-WebUI'...
remote: Enumerating objects: 5622, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 5622 (delta 135), reused 111 (delta 99), pack-reused 5446 (from 2)
Receiving objects: 100% (5622/5622), 1.54 MiB | 26.23 MiB/s, done.
Resolving deltas: 100% (3735/3735), done.
/content/Whisper-WebUI
  Cloning https://github.com/jhj0517/jhj0517-whisper.git to /tmp/pip-req-build-rj9k4f8y
  Running command git clone --filter=blob:none --quiet https://github.com/jhj0517/jhj0517-whisper.git /tmp/pip-req-build-rj9k4f8y
  Resolved https://github.com/jhj0517/jhj0517-whisper.git to commit 197244318d5d75d9d195bff0705ab05a591684ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:0

  Cloning https://github.com/jhj0517/ultimatevocalremover_api.git to /tmp/pip-req-build-5xnd90xg
  Running command git clone --filter=blob:none --quiet https://github.com/jhj0517/ultimatevocalremover_api.git /tmp/pip-req-build-5xnd90xg
  Resolved https://github.com/jhj0517/ultimatevocalremover_api.git to commit 3543be1349ce601568787b69cca0c1f8acba7c2e
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#@title # (Optional) Mount Google Drive
#@markdown Uploading large input files directly via UI may consume alot of time because it has to be uploaded in colab's server.
#@markdown <br>This section is for using the input file paths from Google Drive to reduce such file uploading time.
#@markdown <br>For example, you can first upload the input file to Google Drive and use the directroy path in the "Input Folder Path" input, as shown below.

#@markdown ![image](https://github.com/user-attachments/assets/85330905-e3ec-4502-bc4b-b9d1c5b41aa2)

#@markdown <br>And it will mount the output paths to your Google Drive's as well. This section is optional and can be ignored.


# Mount Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')


# Symlink Output Paths for Whisper-WebUI
import os

OUTPUT_DIRECTORY_PATH = '/content/drive/MyDrive/Whisper-WebUI/outputs'  # @param {type:"string"}
local_output_path = '/content/Whisper-WebUI/outputs'
os.makedirs(local_output_path, exist_ok=True)
os.makedirs(OUTPUT_DIRECTORY_PATH, exist_ok=True)

if os.path.exists(local_output_path):
    !rm -r "$local_output_path"

os.symlink(OUTPUT_DIRECTORY_PATH, local_output_path)
!ls "$local_output_path"

Mounted at /content/drive


In [4]:
#@title # (Optional) Configure arguments
#@markdown This section is used to configure some command line arguments.

#@markdown You can simply ignore this section and the default values will be used.

USERNAME = '' #@param {type: "string"}
PASSWORD = '' #@param {type: "string"}
WHISPER_TYPE = 'faster-whisper' # @param ["whisper", "faster-whisper", "insanely_fast_whisper"]
THEME = '' #@param {type: "string"}

arguments = ""
if USERNAME:
  arguments += f" --username {USERNAME}"
if PASSWORD:
  arguments += f" --password {PASSWORD}"
if THEME:
  arguments += f" --theme {THEME}"
if WHISPER_TYPE:
  arguments += f" --whisper_type {WHISPER_TYPE}"


#@markdown If you wonder how these arguments are used, you can see the [Wiki](https://github.com/jhj0517/Whisper-WebUI/wiki/Command-Line-Arguments).

In [ ]:
#@title #Run
#@markdown Once the installation is complete, you can use public URL that is displayed.
if 'arguments' in locals():
  !python app.py --share --colab --allowed_path "['/content/Whisper-WebUI/outputs']"{arguments}
else:
    !python app.py --share --colab --allowed_path "['/content/Whisper-WebUI/outputs']"

2025-04-13 10:14:59.900007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744539300.260184    2179 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744539300.360706    2179 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-13 10:15:01.114364: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Use "faster-whisper" implementation
Device "cuda" is detected
* Running on local URL:  http://127.0.0.1:7860
* Runnin